In [8]:
from geopy.geocoders import ArcGIS
from geopy.exc import GeocoderTimedOut
import pandas as pd


In [9]:
with open('company_table.pkl', 'rb') as f:
    company_table = pd.read_pickle(f)

In [10]:
#Removing empty addresses 
company_table = company_table[(company_table['address']!='N/A') & (company_table['address']!='')]

In [11]:
# Function to geocode addresses and return (latitude, longitude)
def geocode_address(address):
    geolocator = ArcGIS(timeout=10)
    try:
        location = geolocator.geocode(address)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except GeocoderTimedOut:
        return None, None
    except Exception as e:
        print(f"Error geocoding address '{address}': {e}")
        return None, None

In [ ]:
# Applying geocoding to each row in the DataFrame and create 'Location' column
locations = []
for index, row in company_table.iterrows():
    address = row['address']
    lat, lon = geocode_address(address)
    locations.append((lat, lon))
    print(f'Added location {lat}, {lon}')
print('Geocoding is comleted')

In [13]:
company_table['location'] = locations

In [16]:
#converting year_extablished column type to int and filling nans
company_table['year_established'] = pd.to_numeric(company_table['year_established'], errors='coerce')
company_table['year_established'] = company_table['year_established'].fillna(0).astype('int')   

In [17]:
company_table.to_pickle('company_table.pkl')